In [1]:
import sys
import os

# 添加模块搜索路径
module_path = '/home/lzl/code/python/tartanvo_train_implementation'
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import os

os.environ['PYTHONPATH']='/home/lzl/code/python/tartanvo_train_implementation'
import torch
import sys
import toml
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.tensorboard import SummaryWriter   

from Network.VONet import VONet

from utils.train_pose_utils import load_from_file,load_model, save_model, train_pose_batch, validate
import argparse


In [3]:
def load_model(model, optimizer=None, scheduler=None, filepath=""):
    """
    加载模型、优化器和调度器的状态

    参数:
    model (torch.nn.Module): 要加载状态的模型
    optimizer (torch.optim.Optimizer): 要加载状态的优化器
    scheduler (torch.optim.lr_scheduler._LRScheduler): 要加载状态的调度器
    filepath (str): 要加载文件的路径

    返回:
    int: 加载的epoch
    int: 加载的迭代步数
    """
    if filepath=="":
        return 0,0
    checkpoint = torch.load(filepath)
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler is not None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    epoch = checkpoint['epoch']  # 如果epoch没保存，默认值为0
    iteration = checkpoint['iteration']
    print(f"successfully load model from {filepath}")
    return epoch+1, iteration

In [4]:
model = VONet()
model = torch.nn.DataParallel(model)

/home/lzl/miniforge3/envs/311/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


In [5]:
pwcweight = torch.load('/home/lzl/code/python/tartanvo_train_implementation/models/pwcdcnet/pwc_net_chairs.pth.tar')
flowposeweight = torch.load('/home/lzl/code/python/tartanvo_train_implementation/models/POSEONLY_NORCR/flowpose_model_iteration_99844.pth')
strippedFlowPose = {k.replace('module.',''):v for k,v in flowposeweight['model_state_dict'].items()}
model.module.flowNet.load_state_dict(pwcweight)
model.module.flowPoseNet.load_state_dict(strippedFlowPose)
torch.save({
    'model_state_dict': model.module.state_dict(),
    'optimizer_state_dict': None,
    'scheduler_state_dict': None,
    'epoch': 0,
    'iteration': 0
},'/home/lzl/code/python/tartanvo_train_implementation/models/whole_no_rcr/init.pth')

<All keys matched successfully>

In [7]:
pwcweight = torch.load('/home/lzl/code/python/tartanvo_train_implementation/models/pwcdcnet/pwc_net_chairs.pth.tar')
flowposeweight = torch.load('/home/lzl/code/python/tartanvo_train_implementation/models/POSEONLY_RCR/flowpose_model_iteration_99844.pth')
strippedFlowPose = {k.replace('module.',''):v for k,v in flowposeweight['model_state_dict'].items()}
model.module.flowNet.load_state_dict(pwcweight)
model.module.flowPoseNet.load_state_dict(strippedFlowPose)
torch.save({
    'model_state_dict': model.module.state_dict(),
    'optimizer_state_dict': None,
    'scheduler_state_dict': None,
    'epoch': 0,
    'iteration': 0
},'/home/lzl/code/python/tartanvo_train_implementation/models/whole_rcr/init.pth')
